In [11]:
import os

In [16]:
%pwd

'c:\\Users\\Dell\\Downloads\\Rent_Ml_flow_AWS\\Rent_Prediction_Using_ML\\research'

In [17]:
os.chdir("../")

In [18]:
%pwd

'c:\\Users\\Dell\\Downloads\\Rent_Ml_flow_AWS\\Rent_Prediction_Using_ML'

In [39]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: str
    num_of_epochs: int
    lr: float
    target_column: str
    

In [40]:
from src.mlProject.constants import *
from src.mlProject.utils.common import read_yaml, create_directories

In [41]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.NeuralNetwork
        schema =  self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_trainer_config = ModelTrainerConfig(
            root_dir=config.root_dir,
            train_data_path = config.train_data_path,
            test_data_path = config.test_data_path,
            model_name = config.model_name,
            # alpha = params.alpha,
            # l1_ratio = params.l1_ratio,
            num_of_epochs = params.num_of_epochs,
            lr = params.lr,
            target_column = schema.name
            
        )

        return model_trainer_config

In [42]:
import pandas as pd
import os
from src.mlProject import logger
import torch
import torch.nn as nn
import torch.optim as optim
import joblib

In [43]:
class linearRegression(nn.Module): # all the dependencies from torch will be given to this class [parent class] # nn.Module contains all the building block of neural networks:
  def __init__(self,input_dim):
    super(linearRegression,self).__init__()  # building connection with parent and child classes
    self.fc1=nn.Linear(input_dim,64)          # hidden layer 1
    self.fc2=nn.Linear(64,32)                  # hidden layer 2
    self.fc3=nn.Linear(32,16)                   # hidden layer 3
    self.fc4=nn.Linear(16,8)                   #hidden layer 4
    self.fc5=nn.Linear(8,1)                   # last layer

  def forward(self,d):
    out=torch.relu(self.fc1(d))              # input * weights + bias for layer 1
    out=torch.relu(self.fc2(out))            # input * weights + bias for layer 2
    out=torch.relu(self.fc3(out))            # input * weights + bias for layer 3
    out=torch.relu(self.fc4(out))            # input * weights + bias for layer 4
    out=self.fc5(out)                        # input * weights + bias for last layer
    return out                               # final outcome

In [46]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config

    
    def train(self):
        train_data = pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)


        train_x = train_data.drop([self.config.target_column], axis=1)
        test_x = test_data.drop([self.config.target_column], axis=1)
        train_y = train_data[[self.config.target_column]]
        test_y = test_data[[self.config.target_column]]

        # Convert to PyTorch tensors
        X_train = torch.tensor(train_x.values, dtype=torch.float32)
        X_test = torch.tensor(test_x.values, dtype=torch.float32)
        y_train = torch.tensor(train_y.values, dtype=torch.float32)
        y_test = torch.tensor(test_y.values, dtype=torch.float32)

        input_dim=X_train.shape[1]
        torch.manual_seed(42)  # to make initialized weights stable:
        model = linearRegression(input_dim)
        loss=nn.MSELoss() # loss function
        optimizers=optim.Adam(params=model.parameters(),lr=self.config.lr)

        # training the model:
        for i in range(self.config.num_of_epochs):
            # give the input data to the architecure
            y_train_prediction=model(X_train)  # model initializing
            loss_value=loss(y_train_prediction.squeeze(),y_train)   # find the loss function:
            optimizers.zero_grad() # make gradients zero for every iteration so next iteration it will be clear
            loss_value.backward()  # back propagation
            optimizers.step()  # update weights in NN

            # print the loss in training part:
            if i % 10 == 0:
                logger.info(f"[epoch:{i}]: The training loss={loss_value}")

        joblib.dump({'model_state_dict': model.state_dict()}, os.path.join(self.config.root_dir, self.config.model_name))


In [47]:
try:
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer_config = ModelTrainer(config=model_trainer_config)
    model_trainer_config.train()
except Exception as e:
    raise e

[2025-01-16 18:53:09,352: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-01-16 18:53:09,360: INFO: common: yaml file: params.yaml loaded successfully]
[2025-01-16 18:53:09,374: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-01-16 18:53:09,386: INFO: common: created directory at: artifacts]
[2025-01-16 18:53:09,393: INFO: common: created directory at: artifacts/model_trainer]


c:\Users\Dell\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\nn\modules\loss.py:535: UserWarning: Using a target size (torch.Size([9495, 1])) that is different to the input size (torch.Size([9495])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


[2025-01-16 18:53:15,252: INFO: 110644740: [epoch:0]: The training loss=6082070528.0]
[2025-01-16 18:53:47,514: INFO: 110644740: [epoch:10]: The training loss=6009809920.0]
